# Środowisko Google Kubernetes Engine 

Logowanie kontem github. 

## Wykorzystanie Google Storage


#### Projekt
* Wszystki dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy. 
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc. 
#### Kubełek (bucket)
* Kubełek (buckjet) to kontener na pliki/obiekty. 
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!) 
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie.
#### Obiekt 
* obiekty przechowywane w kubełkach 
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne 

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`: 

##### Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls` 
* udostępnianie - `iam`

##### Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie między kubełkami - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`


#### Operacje na kubełkach

In [1]:
! gsutil ls

gs://bdg-lab-adys/
gs://bdg-lab-akozlow4/
gs://bdg-lab-alysik1/
gs://bdg-lab-amysliw1/
gs://bdg-lab-apeszek/
gs://bdg-lab-aputowsk/
gs://bdg-lab-astolins/
gs://bdg-lab-bstypins/
gs://bdg-lab-dbartos1/
gs://bdg-lab-dszulc/
gs://bdg-lab-grodewal/
gs://bdg-lab-jczarnoc/
gs://bdg-lab-kjastrze/
gs://bdg-lab-kkrol2/
gs://bdg-lab-kkromke/
gs://bdg-lab-kpapaj/
gs://bdg-lab-ksotomsk/
gs://bdg-lab-ktora/
gs://bdg-lab-kwojszk1/
gs://bdg-lab-mdrabeck/
gs://bdg-lab-mendzel/
gs://bdg-lab-mkrajews/
gs://bdg-lab-mlech/
gs://bdg-lab-mlipiec/
gs://bdg-lab-mnizinsk/
gs://bdg-lab-mptaszek/
gs://bdg-lab-mwietes1/
gs://bdg-lab-mwronsk2/
gs://bdg-lab-pkaluski/
gs://bdg-lab-pkluczwa/
gs://bdg-lab-pstanis2/
gs://bdg-lab-pzolynia/
gs://bdg-lab-rlasota/
gs://bdg-lab-rsarna/
gs://bdg-lab-schojnac/
gs://bdg-lab-sgdaniec/
gs://bdg-lab-skosek/
gs://bdg-lab-srychlic/
gs://ds_labs_agaszmurlo/
gs://test_agaszmurlo/


In [6]:
! echo $USER  # wyswietlenie nazwy uzytkownika

ds-lab-testuser1


In [16]:
! gsutil mb gs://bucket-$USER  # stworzenie bucketu

Creating gs://bucket-ds-lab-testuser1/...


In [18]:
! gsutil ls -L -b gs://bucket-$JUPYTERHUB_USER # listowanie zawartości 

gs://bucket-ds-lab-testuser1/ :
	Storage class:			STANDARD
	Location type:			multi-region
	Location constraint:		US
	Versioning enabled:		None
	Logging configuration:		None
	Website configuration:		None
	CORS configuration: 		None
	Lifecycle configuration:	None
	Requester Pays enabled:		None
	Labels:				None
	Default KMS key:		None
	Time created:			Thu, 10 Dec 2020 15:47:00 GMT
	Time updated:			Thu, 10 Dec 2020 15:47:00 GMT
	Metageneration:			1
	Bucket Policy Only enabled:	False
	ACL:				
	  [
	    {
	      "entity": "project-owners-826258521081",
	      "projectTeam": {
	        "projectNumber": "826258521081",
	        "team": "owners"
	      },
	      "role": "OWNER"
	    },
	    {
	      "entity": "project-editors-826258521081",
	      "projectTeam": {
	        "projectNumber": "826258521081",
	        "team": "editors"
	      },
	      "role": "OWNER"
	    },
	    {
	      "entity": "project-viewers-826258521081",
	      "projectTeam": {
	        "projectNumber": "826258521081",
	

In [19]:
! gsutil du -s  gs://bucket-$USER # ile zajmuje przestrzeni?

0            gs://bucket-ds-lab-testuser1


#### Operacje na zawartości kubełków

In [20]:
! gsutil ls -r gs://bucket-$USER/ # listowanie zawartosci kubełka

In [ ]:
! gsutil cp ~/ds-notebooks/README.md gs://bucket-$USER  # upload obiektu do kubełka

In [22]:
! gsutil ls -r gs://bucket-$USER # listowanie zawartości

In [ ]:
! gsutil stat gs://bucket-$USER/README.md # metadane obiekty w kubełku

In [ ]:
! gsutil iam ch allUsers:objectViewer gs://bucket-$USER # dodanie uprawnien do odczytu

Po wykonaniu tego polecenia nasz kubełek staje się publiczny i możemy się do niego 
http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA. Zawartość pliku można odczytać poprzez http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA/NAZWA_OBIEKTU

In [ ]:
! gsutil iam ch -d allUsers:objectViewer gs://bucket-$USER # usuniecie uprawnien do odczytu

Teraz można ponownie zweryfikowac możliwość publicznego odczytu danych z kubełka.

#### Przeniesienie do docelowego kubełka

In [ ]:
%%bash
gsutil cp -r gs://bucket-$USER gs://DOCELOWY_BUCKET
gsutil rm -r gs://bucket-$USER

## Apache Spark na GKE

Podobnie jak na klastrze nawiążemy połączenie z sesją Spark i odczytamy dane z pliku udostępnionego na GCS.

In [1]:
import os
user_name = os.environ.get('USER')
print(user_name)

ds-lab-testuser1


In [ ]:
import random
ui_port = random.randint(4000,4999)
print(ui_port)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.master('yarn-client') \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.config('spark.ui.port',f'{ui_port}') \
.appName(f'ds-{user_name}') \
.getOrCreate()

In [4]:
path = f'/user/{user_name}/external/data/brca.txt' # todo


In [ ]:
df = spark.read.load(path, format="csv", sep="\t", inferSchema="true", header="true")

# df.registerTempTable(f"{table_name}")

In [2]:
type (df)

NameError: name 'df' is not defined

In [3]:
df.printSchema()

NameError: name 'df' is not defined

Powtórzenie operacji na DataFrame jak na klastrze Hadoop.